In [1]:
from os import path, listdir
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

/home/users/mwlw3/miniconda3/envs/AQmort/lib/python3.9/site-packages/pyproj/__init__.py:89: UserWarning: pyproj unable to set database path.
  _pyproj_global_context_initialize()


In [ ]:
folderpath = "/neodc/sentinel5p/data/L2_NO2/v2.2/2021/11/05/"

In [ ]:
listdir(folderpath)[:5]

In [2]:
class S5P_Filename():
    def __init__(self, filename):
        self.filename = filename
        self.mission = filename[0:3]
        self.stream = filename[4:8]
        self.product = filename[9:19]
        self.granule_start = filename[20:35]
        self.granule_end = filename[36:51]
        self.orbit = filename[52:57]
        self.collection = filename[58:60]
        self.processor_version = filename[61:67]
        self.processing_time = filename[68:83]
        self.extension = filename[83:86]

In [ ]:
filename = S5P_Filename("S5P_NRTI_L2__NO2____20211105T200216_20211105T200716_21060_02_020200_20211105T215352.nc")

In [ ]:
files = []

for file in listdir(folderpath):
    file = S5P_Filename(file)
    if file.extension == ".nc":
        files.append(path.join(folderpath, file.filename))

In [ ]:
ds = xr.open_dataset(path.join(folderpath, listdir(folderpath)[0]), group="PRODUCT") # might want to use open_mfdataset() for multiple files

In [ ]:
ds = xr.open_mfdataset(files, group="PRODUCT")

In [ ]:
ds.nitrogendioxide_tropospheric_column.plot()

In [ ]:
lats = ds.latitude[:][0,:,:]
lons = ds.longitude[:][0,:,:]
no2 = ds.nitrogendioxide_tropospheric_column[0,:,:]
no2_units = ds.nitrogendioxide_tropospheric_column_precision.units

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()

plt.pcolor(lons, lats, no2, cmap="hot")
plt.show()

Let me try to open and plot all the data from the specified date, so I can see a broader spatial region.

In [ ]:
datasets = []

for file in listdir(folderpath):
    file = S5P_Filename(file)
    if file.extension == ".nc":
        ds = xr.open_dataset(path.join(folderpath, file.filename), group="PRODUCT")
        datasets.append(ds.copy())

ds = xr.concat(datasets, "scanline")

In [ ]:
ds.nitrogendioxide_tropospheric_column.plot()

In [ ]:
lats = ds.latitude[:][0,:,:]
lons = ds.longitude[:][0,:,:]
no2 = ds.nitrogendioxide_tropospheric_column[0,:,:]

In [ ]:
plt.pcolor(lons, lats, no2, cmap="hot")
plt.show()

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()

plt.contourf(lons, lats, no2, transform=ccrs.PlateCarree())
plt.show()

Try with all files from the same month:

In [15]:
files = []

folderpath = "/neodc/sentinel5p/data/L2_NO2/v2.2/2021/10/"
for day_folder in listdir(folderpath)[1:]:
    for file in listdir(path.join(folderpath, day_folder)):
        file = S5P_Filename(file)
        if file.extension == ".nc":
            files.append(path.join(folderpath, day_folder, file.filename))

In [16]:
len(files)

5903

In [ ]:
datasets = []
for file in files:
    ds = xr.open_dataset(file, group="PRODUCT")
    datasets.append(ds.copy())
    
ds = xr.concat(datasets, "scanline")